# CCIC GridSat time series

This notebook computes cloud amount and TIWP time series from the CCIC GridSat record.

In [ ]:
from pathlib import Path

import cmocean
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

plt.style.use("../ccic.mplstyle")

In [ ]:
DATA_PATH = Path("/scratch/ccic_record/data/")
gridsat_files = sorted(list((DATA_PATH / "ccic" / "gridsat").glob("*.nc")))
print(f"Found {len(gridsat_files)} files.")

In [ ]:
mask = xr.load_dataset("/scratch/ccic_record/data/mask_24.nc").mask.data

The code below iterates over all monthly CCIC GridSat files and calculates mean field of cloud amount (``ca``) and total ice water path (``tiwp``) as well as time series of their area-weighted means and masked means.

In [ ]:
from tqdm import tqdm

tiwp_sum = None
tiwp_cnt = None
tiwp_mean = None
tiwp_mean_masked = None
ca_sum = None
ca_cnt = None
ca_mean = None
ca_mean_masked = None
valid_frac = None
valid_frac_masked = None
time = []

for path in tqdm(gridsat_files):
    with xr.open_dataset(path) as input_data:

        tiwp = input_data.tiwp.data[0]
        ca = input_data.cloud_prob_2d.data[0]
        time.append(input_data.month.data[0])
        tiwp_0 = np.nan_to_num(tiwp, nan=0.0, copy=True)
        ca_0 = np.nan_to_num(ca, nan=0.0, copy=True)

        weights = np.cos(np.deg2rad(input_data.latitude.data[..., None]))
        weights = np.broadcast_to(weights, tiwp.shape)
        weights_sum = weights.sum()
        weights_masked = weights * mask
        weights_masked_sum = weights_masked.sum()

        if tiwp_sum is None:
            tiwp_sum = tiwp_0
            tiwp_cnt = np.isfinite(tiwp).astype("float32")
            ca_sum = ca_0
            ca_cnt = np.isfinite(ca).astype("float32")
            tiwp_mean = [(tiwp_0 * weights).sum() / (weights * np.isfinite(tiwp)).sum()]
            tiwp_mean_masked = [
                (tiwp_0 * weights_masked).sum()
                / (weights_masked * np.isfinite(tiwp)).sum()
            ]
            ca_mean = [(ca_0 * weights).sum() / (weights * np.isfinite(ca)).sum()]
            ca_mean_masked = [
                (ca_0 * weights * mask).sum() / (weights_masked * np.isfinite(ca)).sum()
            ]
            valid_frac = [(weights * np.isfinite(ca)).sum() / weights_sum]
            valid_frac_masked = [(weights * np.isfinite(ca)).sum() / weights_masked_sum]
        else:
            tiwp_sum += tiwp_0
            tiwp_cnt += np.isfinite(tiwp).astype("float32")
            ca_sum += ca_0
            ca_cnt += np.isfinite(ca).astype("float32")
            tiwp_mean += [
                (tiwp_0 * weights).sum() / (weights * np.isfinite(tiwp)).sum()
            ]
            tiwp_mean_masked += [
                (tiwp_0 * weights_masked).sum()
                / (weights_masked * np.isfinite(tiwp)).sum()
            ]
            ca_mean += [(ca_0 * weights).sum() / (weights * np.isfinite(ca)).sum()]
            ca_mean_masked += [
                (ca_0 * weights * mask).sum() / (weights_masked * np.isfinite(ca)).sum()
            ]
            valid_frac += [(weights * np.isfinite(ca)).sum() / weights_sum]
            valid_frac_masked += [
                (weights * np.isfinite(ca)).sum() / weights_masked_sum
            ]

        latitude = input_data.latitude.data
        longitude = input_data.longitude.data

time = np.array(time)

results = xr.Dataset(
    {
        "tiwp": (("latitude", "longitude"), tiwp_sum / tiwp_cnt),
        "ca": (("latitude", "longitude"), ca_sum / ca_cnt),
        "tiwp_mean": (("time",), np.array(tiwp_mean)),
        "tiwp_mean_masked": (("time",), np.array(tiwp_mean_masked)),
        "ca_mean": (("time",), np.array(ca_mean)),
        "ca_mean_masked": (("time",), np.array(ca_mean_masked)),
        "valid_frac": (("time",), np.array(valid_frac)),
        "valid_frac_masked": (("time",), np.array(valid_frac_masked)),
    }
)

In [ ]:
results.to_netcdf("/scratch/ccic_record/data/timeseries/ccic_gridsat.nc")